In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from itertools import product
from dataclasses import dataclass
from collections import namedtuple, defaultdict
from tasks.markov import *
from models.base_models import *
from models.pos_encoder import *
from models.forward_tracer import *
from tasks.test_markov import *
from config import *
from train import *
import figures.plot as plot
from figures.ood_plots import *
from util import *
import seaborn as sns
import torch.utils.benchmark as benchmark
from tqdm.notebook import tqdm, trange
import os
import re

from figures.head_view import *

torch.set_printoptions(precision=3, sci_mode=False)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
SEQ_LEN, VOC_SIZE, BATCH_SIZE, alpha = 128, 5, 32, 1
pos_enc="rotary"
hidden_dim=128
mlp_layers=None
epochs=10000
num_layers=2

flash = False
    
if pos_enc in ["rotary", "abs"]:
    flash = True

if mlp_layers is None:
    mlp_layers = [i for i in range(num_layers)]
    
mlp = [False] * num_layers
for i in mlp_layers:
    mlp[i] = True 
mlp = tuple(mlp)

config = Config(
        emb_dim=hidden_dim,
        num_layers=num_layers,
        num_heads=tuple([1]*num_layers),
        identity_query=False,
        seq_len=SEQ_LEN,
        vocab_size=VOC_SIZE,
        batch_size=BATCH_SIZE,
        num_epochs=epochs,
        eval_iter=1000,
        pos_enc=pos_enc,
        pos_max_len=SEQ_LEN,
        get_attn=500,
        get_checkpoints=200,
        mlp=mlp,
        activation=mlp,
        flash=flash, # Use rotary
        ff_dim=hidden_dim,
        layer_norm=False,
        ngram=2,
        learning_rate=1e-4,
        task_name="icl-mc"
    )

sampler_config = MarkovSamplerConfig(seq_len=SEQ_LEN, vocab_size=VOC_SIZE, order=1, alpha=alpha,
                                     batch_size=BATCH_SIZE, task_name="icl-mc")

model = Transformer(config)

In [15]:
get_layers(model)

ModuleList(
  (0-1): 2 x TFBlock(
    (MHA): MultiHeadAttention(
      (query): Linear(in_features=128, out_features=128, bias=False)
      (key): Linear(in_features=128, out_features=128, bias=False)
      (value): Linear(in_features=128, out_features=128, bias=False)
      (out): Linear(in_features=128, out_features=128, bias=False)
    )
    (ln1): Identity()
    (mlp): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): ReLU()
      (2): Linear(in_features=128, out_features=128, bias=True)
    )
    (ln2): Identity()
  )
)